In [1]:
import numpy as np
import os
import math

In [2]:
def mom(arr):
    return math.sqrt(arr[3]**2+arr[1]**2+arr[2]**2)
    
def theta(arr):
    return math.acos(arr[3]/mom(arr))

def phi(arr):
    return math.atan(arr[0]/arr[1])

def cos_theta(arr):
    return arr[3]/mom(arr)

def theta_dif(arr1,arr2):
    return math.acos(dp(arr1[1:4],arr2[1:4])/mom(arr1)/mom(arr2))

#Expect each element of arr to be 4 elements. Expects px py pz E.
def invariant_mass(arr):
    tot = [0,0,0,0]
    for line in arr:
        for i in range(4):
            tot[i]+=line[i]
    return tot[0]**2-tot[3]**2-tot[1]**2-tot[2]**2

def cos_angle_between(arr1,arr2):
    return (arr1[3]*arr2[3]+arr1[1]*arr2[1]+arr1[2]*arr2[2])/mom(arr1)/mom(arr2)

In [3]:
def dp(a1,a2):
    tot=0
    for i in range(len(a1)):
        tot+=a1[i]*a2[i]
    return tot
def sub(a1,a2):
    return [a1[i]-a2[i] for i in range(len(a1))]
def add(a1,a2):
    return [a1[i]+a2[i] for i in range(len(a1))]
def mul(c,a1):
    return [c*a1[i] for i in range(len(a1))]
def plane_cross(p,o,r,n):
    bot=dp(n,p)
    if bot == 0:
        return -1
    a = dp(n,sub(r,o))/bot
    R=sub(r,add(o,mul(a,p)))
    return dp(R,R)**0.5

In [4]:
def record_list(outfile,data):
    with open(outfile,'w') as w:
        for line in data:
            tmpstr=""
            for elem in line:
                tmpstr+=str(elem)+' '
            tmpstr=tmpstr[:-1]
            tmpstr+='\n'
            w.write(tmpstr)

In [5]:
def Event_Parser(event_file, cut):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        run_num=''
        for line in dat:
            line=line.split()
            if len(line)==2 and line[0]=="Run":
                run_num=line[1]                
                break
    dat2 = [line.split() for line in dat]
    sum_line= dat2[-1]
    electron = [[float(line[i]) for i in range(1,len(line))] for line in dat2 if len(line)>1 and line[0]=='electron']
    mv = sum_line[1]
    mx = sum_line[2]
    nevents = float(sum_line[3])
    POT = float(sum_line[7])
    alpha_D=float(sum_line[4])
    eps=float(sum_line[5])
    
    return [float(mv),float(mx),nevents*cut(electron,POT),alpha_D,eps,sum_line]

In [6]:
def CHARM2_cut(data,POT):
    charm2_cut=[theta(line)**2*line[3] for line in data]
    cha_pas=0
    for i in charm2_cut:
        if i<0.001:
            cha_pas+=1
    
    charm2_pot=2.5e19
    if cha_pas is 0:
        charm2_eff=0
    else:
        charm2_eff=charm2_pot/POT*cha_pas/len(charm2_cut)
     
    return charm2_eff

In [7]:
def Minerva_cut(data,POT):
    minerva_cut = [theta(line)**2*line[3] for line in data]
    min_pas=0
    for i in minerva_cut:
        if i<0.0032:
            min_pas+=1

    minerva_POT=3.43e20
    if min_pas is 0:
        minerva_eff=0
    else:
        minerva_eff=minerva_POT/POT*min_pas/len(minerva_cut)
        
    return minerva_eff

In [8]:
def NOvA_cut(data,POT):
    nova_cut = [theta(line)**2*line[3] for line in data]
    nov_pas=0
    for i in nova_cut:
        if i<0.005:
            nov_pas+=1

    nov_POT=2.97e20
    if nov_pas is 0:
        nova_eff=0
    else:
        nova_eff=nov_POT/POT*nov_pas/len(nova_cut)
        
    return nova_eff

In [7]:
def bebc_cut(data,POT):
    bebc_cut=[[line[3],theta(line)**2*line[3]] for line in data]
    beb_pas=0
    for i in bebc_cut:
        if i[0]>0.5 and i[1]<2*511e-6:
            beb_pas+=1
    
    #bebc_pot=2.5e19
    if beb_pas == 0:
        bebc_eff=0
    else:
        bebc_eff=beb_pas/len(bebc_cut)
     
    return bebc_eff

In [10]:
mini_numi_angle=0.10472
mini_cos_cut=0.99;
mini_POT=6e20
mnx=math.sin(mini_numi_angle);mny=0;mnz=math.cos(mini_numi_angle)
mini_arr=[0,mnx,mny,mnz]
def mini_numi_cut(data,POT):
    mini_cut=[cos_angle_between(x,mini_arr) for x in data]
    mini_pas=0
    for i in mini_cut:
        if i>mini_cos_cut:
            mini_pas+=1

    if mini_POT is 0:
        mini_eff=0
    else:
        mini_eff=mini_POT/POT*mini_pas/len(mini_cut)
    
    return mini_eff

In [11]:
def Intersection_Parser(event_file):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
        Total_Line=''
        for line in dat:
            line=line.split()
            if line[0]=="Total":
                Total_Line=line
                break
    dat2 = [line.split() for line in dat]
    mass=dat2[1][7]
    Total_Line=Total_Line+[mass]
    dat2 = [Total_Line]+dat2[:-1]
    return dat2

In [12]:
def Clean_Event_File(event_file):
    with open(event_file) as ef:
        dat=ef.read().splitlines()
    dat2 = [line.split() for line in dat]
    Total_Line=dat2[-1]
    dat2 = [Total_Line]+dat2[:-1]
    return dat2

In [11]:
file_list=np.sort(os.listdir("~/Code/cpp/bdnmc_events/BEBC/RUN_2"))

FileNotFoundError: [Errno 2] No such file or directory: '~/Code/cpp/bdnmc_events/BEBC/RUN_2'

In [39]:
file_list=[x for x in file_list if x.count("minerva_")>0]

array(['DUNE_HPg.dat', 'DUNE_HPg_1000_0.001.dat',
       'DUNE_HPg_100_0.001.dat', 'DUNE_HPg_200_0.001.dat',
       'DUNE_HPg_300_0.001.dat', 'DUNE_HPg_400_0.001.dat',
       'DUNE_HPg_500_0.001.dat', 'DUNE_HPg_50_0.001.dat',
       'DUNE_HPg_600_0.001.dat', 'DUNE_HPg_800_0.001.dat',
       'numi_YuDai.dat', 'numi_YuDai_decay_Events_1000_1e-08.dat',
       'numi_YuDai_decay_Events_100_1e-08.dat',
       'numi_YuDai_decay_Events_1250_1e-08.dat',
       'numi_YuDai_decay_Events_1500_1e-08.dat',
       'numi_YuDai_decay_Events_2000_1e-08.dat',
       'numi_YuDai_decay_Events_200_1e-08.dat',
       'numi_YuDai_decay_Events_300_1e-08.dat',
       'numi_YuDai_decay_Events_500_1e-08.dat',
       'numi_YuDai_decay_Events_50_1e-08.dat',
       'numi_YuDai_decay_Events_700_1e-08.dat',
       'numi_YuDai_decay_Events_900_1e-08.dat'], dtype='<U38')

In [41]:
file_list=np.sort(os.listdir("Decay_Events/"))
file_list=[x for x in file_list if x.count("DUNE_HPg_")>0]
eventlist=[Intersection_Parser('Decay_Events/'+file) for file in file_list[:]]
#np.savetxt("minerva_events.dat",eventlist)

In [19]:
file_list=np.sort(os.listdir("Claudia/"))
file_list=[x for x in file_list if x.count("charm2_")>0]
charm2eventlist=[Event_Parser('Claudia/'+file,CHARM2_cut) for file in file_list[:]]
#record_list("Claudia/charm2elec.dat",eventlist)
#np.savetxt("charm2_events.dat",eventlist)

In [20]:
tot_tab=[]
for x in charm2eventlist:
    x[-1][3]=x[2]*1
    x[-1][0]="Total"
    tot_tab+=[x[-1]]
record_list("Claudia/charm2elec.dat",tot_tab)      

In [14]:
file_list=np.sort(os.listdir("Claudia/"))
file_list=[x for x in file_list if x.count("bebc_")>0]
bebceventlist=[Event_Parser('Claudia/'+file,bebc_cut) for file in file_list[:]]

In [15]:
tot_tab=[]
for x in bebceventlist:
    x[-1][3]=x[2]*1
    x[-1][0]="Total"
    tot_tab+=[x[-1]]
record_list("Claudia/bebcelec.dat",tot_tab)

In [40]:
file_list=np.sort(os.listdir("Claudia_MB_Numi/"))
file_list=[x for x in file_list if x.count("MB_Numi_scatter")>0]
mbeventlist=[Event_Parser('Claudia_MB_Numi/'+file,mini_numi_cut) for file in file_list[:]]

In [41]:
tot_tab=[]
for x in mbeventlist:
    x[-1][3]=x[2]*0.35
    x[-1][0]="Total"
    tot_tab+=[x[-1]]
record_list("Claudia_MB_Numi/numi_elec_sum.dat",tot_tab)

In [29]:
file_list=np.sort(os.listdir("Claudia/"))
file_list=[x for x in file_list if x.count("minerva_")>0]
minervaeventlist=[Event_Parser('Claudia/'+file,Minerva_cut) for file in file_list[:]]

In [31]:
tot_tab=[]
for x in minervaeventlist:
    x[-1][3]=x[2]*0.709
    x[-1][0]="Total"
    tot_tab+=[x[-1]]
record_list("Claudia/minerva_elec.dat",tot_tab)

In [32]:
file_list=np.sort(os.listdir("Claudia/"))
file_list=[x for x in file_list if x.count("nova_")>0]
novaeventlist=[Event_Parser('Claudia/'+file,NOvA_cut) for file in file_list[:]]

In [33]:
tot_tab=[]
for x in novaeventlist:
    x[-1][3]=x[2]*0.5
    x[-1][0]="Total"
    tot_tab+=[x[-1]]
record_list("Claudia/nova_elec.dat",tot_tab)

In [70]:
file_list=np.sort(os.listdir("Decay_Events_2/"))
file_list=[x for x in file_list if x.count("DUNE_HPg_")>0]
eventlist=[Clean_Event_File('Decay_Events_2/'+file) for file in file_list[:]]
#np.savetxt("minerva_events.dat",eventlist)

In [71]:
for i in range(len(file_list)):
    record_list('YuDai_project/DUNE_decays/'+file_list[i],eventlist[i])

In [9]:
meta=0.547862

In [10]:
charm2_file=np.loadtxt("data/particle_list_charm2.dat");
charm2_file=charm2_file[:100000]

In [13]:
charm2_file_eta=[[x[0],x[1],x[2],math.sqrt(x[0]**2+x[1]**2+x[2]**2+meta**2)] for x in charm2_file];

In [18]:
math.sqrt(charm2_file_eta[1][3]**2-charm2_file_eta[1][0]**2-charm2_file_eta[1][2]**2-charm2_file_eta[1][1]**2)

0.5478620000001176

In [14]:
np.savetxt("charm2_eta.dat",charm2_file_eta)